In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import nltk
import re
import os
from tqdm.notebook import tqdm

/Users/amikano/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cwd = os.getcwd()

os.chdir(cwd+"/data/")

In [3]:
def parser(html_file):
    
    with open(html_file) as html:
        soup = BeautifulSoup(html, 'html.parser')
        
    if len(soup.find_all("ul", id="theatre-details")) != 0:
        original_work = soup.find_all("ul", id="theatre-details")[0].get_text().split("\n")[1]
    else:
        original_work = soup.find_all("a")[1].get_text()
    
    title = soup.find_all("title")[0].get_text()
    author = soup.find_all("meta", attrs={"name":"author"})[0]["content"]
    
    overall_rating_index = str(soup.find("table", id="ratings-stars")).split("<tr>")[-1].index("title=")
    overall_rating = str(soup.find("table", id="ratings-stars")).split("<tr>")[-1][overall_rating_index+len("title="):overall_rating_index+len("title=")+3]
    overall_rating = re.sub(r'[^\w\s]', '', overall_rating)
    
    content_list = [p.get_text().strip() for p in soup.find_all("p")] 
    for c in content_list.copy():
        if c[:1] == "\n":
            content_list.remove(c)
        if c[:7] == "Photos:":
            content_list.remove(c)
        if "www." in c:
            content_list.remove(c)
        if c[:7] == "Discuss":
            content_list.remove(c)
        if "To reserve tickets, please call" in c:
            content_list.remove(c)
        if "Reservations:" in c:
            content_list.remove(c)
        if "Reviews on this site are subject to" in c:
            content_list.remove(c)
        if "Click to listen to" in c:
            content_list.remove(c)
        if "This review also appears on" in c:
            content_list.remove(c)
        if "{ijseo_redirect id=1}" in c:
            content_list.remove(c)
        if "Buy this book on Amazon.com" in c:
            content_list.remove(c)
        
    content = " ".join([s for s in content_list]).replace("\n", "").replace("\\", "").replace("/", "").replace("  ", " ")
    
    if content == "":
        content_list = [p.get_text().strip() for p in soup.find_all("span", {"class": "reviewstext"})]
        if len(content_list) > 0:
            content = " ".join(content_list).replace("\n", "").replace("\\", "").replace("/", "").replace("  ", " ")
        
        else:
            content_list = [p.get_text().strip() for p in soup.find_all("div", {"class": "jrListingFulltext"})]
            if len(content_list) > 0:
                content = " ".join(content_list).replace("\n", "").replace("\n", "").replace("\\", "").replace("/", "").replace("  ", " ")
    
    content = re.sub(r',|"|”|“|\(|\)|’|‘|\#|\$|£|&', '', content)
    content = re.sub(r'\-|\―|\/|\+|…|\—|–', ' ', content)
    content = content.lower()
    
    character_names_tn = r'orsino|sebastian|antonio|valentine|andrew|cesario|curio|toby|malvolio|fabian|feste|olivia|viola|maria'
    character_names_aw = r'lafew|parolles|rinaldo|lavatch|helena|diana|mariana|violenta'
    character_names_asyl = r'amiens|jaques|charles|oliver|orlando|adam|dennis|touchstone|martext|corin|silvius|hymen|rosalind|celia|phebe|audrey'
    character_names_ham = r'claudius|hamlet|polonius|horatio|laertes|voltemand|cornelius|rosencrantz|guildenstern|osric|marcellus|barnado|francisco|reynaldo|fortinbras|gertrude|ophelia'
    character_names_lll = r'ferdinand|berowne|longaville|dumaine|boyet|marcade|adriano|armado|nathaniel|holofernes|costard|forester|rosaline|maria|katherine|jaquenetta|blackamoors'
    character_names_mac = r'duncan|malcolm|donalbain|macbeth|banquo|macduff|lennox|rosse|menteth|angus|cathness|fleance|siward|seyton|hecat'
    character_names_mnd = r'theseus|egeus|lysander|demetrius|philostrate|quince|starveling|hippolyta|hermia|helena|oberon|titania|puck|peaseblossom|cobweb|mustardseed'
    character_names_rj = r'escalus|paris|romeo|juliet|montague|capulet|mercutio|benvolio|tybalt|petruchio|lawrence|john|balthasar|abram|sampson|gregory|peter|anthony|potpan'
    character_names_oth = r'brabantio|gratiano|lodovico|othello|cassio|lago|roderigo|montano|desdemona|emilia|bianca'
    character_names_jc = r'julius|caesar|octavius|mark|antony|lepidus|cicero|publius|popilius|brutus|cassius|casca|trebonius|caius|ligarius|decius|metellus|cimber|cinna|flavius|murellus|artemidorus|cnidos|soothsayer|lucilius|titinius|messala|volumnius|varrus|clitus|claudio|strato|lucius|dardanius|pindarus|calphurnia|cobbler|portia'
    
    character_names = '|'.join([character_names_tn, character_names_aw, character_names_asyl,
                               character_names_ham, character_names_lll, character_names_mac,
                               character_names_mnd, character_names_rj, character_names_oth,
                               character_names_jc])
    filtered_words = "|".join([character_names, "shakespeare|shakespeares|william|williams"])
    
    content = re.sub(filtered_words, '', content)
    
    content_df_sent = pd.DataFrame({"review_id" : [title],
                               'content' : content})['content']\
                            .str.split(r'[.!?:;]+', expand=True).stack().to_frame('sent_str')
    content_df_sent['sent_str'] = content_df_sent['sent_str'].str.strip()
    content_df_sent = content_df_sent[~content_df_sent['sent_str'].str.match(r'^\s*$')]
    content_df_sent.index.names = ['review_id', 'sent_id']
    
    content_df_token = content_df_sent['sent_str'].str.split(r"[\s',-]", expand=True).stack().to_frame('term_str')
    content_df_token['term_str'] = content_df_token['term_str'].str.strip()
    content_df_token = content_df_token[~content_df_token['term_str'].str.match(r'^\s*$')]
    content_df_token.index.names = ['review_id', 'sent_id', 'token_id']
    
    content_df_token = content_df_token.reset_index()
    content_df_token.review_id = [title for i in range(content_df_token.shape[0])]
    content_df_token = content_df_token.set_index(['review_id', 'sent_id', 'token_id'])
    
    content_df_token['pos_tuple'] = nltk.pos_tag(content_df_token.term_str)
    
    
    #------ LIB
    lib = pd.DataFrame({"Original Work":[original_work],
                          "Review Title":[title],
                          "Review Author":[author],
                          "Content":[content],
                          "Overall Rating":[overall_rating]})
    
    return content_df_token, lib

In [4]:
d, l = parser('review1.html')
d.head(20)

term_str  \
review_id                                          sent_id token_id               
Twelfth Night is a New Perspective at the National 0       0             golden   
                                                           1              brown   
                                                           2             leaves   
                                                           3           decorate   
                                                           4                the   
                                                           5              edges   
                                                           6                 of   
                                                           7                the   
                                                           8              stage   
                                                           9                 in   
                                                           11             halls   
                                                           12        production   
                                                           13                of   
                                                           14           twelfth   
                                                           15             night   
                                                           16               for   
                                                           17               the   
                                                           18          national   
                                                           19           theatre   
                                                   1       0                the   

                                                                            pos_tuple  
review_id                                          sent_id token_id                    
Twelfth Night is a New Perspective at the National 0       0             (golden, JJ)  
                                                           1              (brown, NN)  
                                                           2            (leaves, VBZ)  
                                                           3          (decorate, VBP)  
                                                           4                (the, DT)  
                                                           5             (edges, NNS)  
                                                           6                 (of, IN)  
                                                           7                (the, DT)  
                                                           8              (stage, NN)  
                                                           9                 (in, IN)  
                                                           11            (halls, NNS)  
                                                           12        (production, NN)  
                                                           13                (of, IN)  
                                                           14           (twelfth, JJ)  
                                                           15             (night, NN)  
                                                           16               (for, IN)  
                                                           17               (the, DT)  
                                                           18          (national, JJ)  
                                                           19           (theatre, NN)  
                                                   1       0                (the, DT)

In [5]:
num_of_file = len(os.listdir())-1

DOC = pd.DataFrame()
LIB = pd.DataFrame()

for i in tqdm(range(1, num_of_file)):
    
    filename = "review" + str(i) + ".html"
    
    doc, lib = parser(filename)
    
    DOC = pd.concat([DOC, doc])
    
    LIB = pd.concat([LIB, lib])

DOC["pos"] = [i[1] for i in list(DOC["pos_tuple"])]    

DOC.head(20)

  0%|          | 0/292 [00:00<?, ?it/s]

term_str  \
review_id                                          sent_id token_id               
Twelfth Night is a New Perspective at the National 0       0             golden   
                                                           1              brown   
                                                           2             leaves   
                                                           3           decorate   
                                                           4                the   
                                                           5              edges   
                                                           6                 of   
                                                           7                the   
                                                           8              stage   
                                                           9                 in   
                                                           11             halls   
                                                           12        production   
                                                           13                of   
                                                           14           twelfth   
                                                           15             night   
                                                           16               for   
                                                           17               the   
                                                           18          national   
                                                           19           theatre   
                                                   1       0                the   

                                                                            pos_tuple  \
review_id                                          sent_id token_id                     
Twelfth Night is a New Perspective at the National 0       0             (golden, JJ)   
                                                           1              (brown, NN)   
                                                           2            (leaves, VBZ)   
                                                           3          (decorate, VBP)   
                                                           4                (the, DT)   
                                                           5             (edges, NNS)   
                                                           6                 (of, IN)   
                                                           7                (the, DT)   
                                                           8              (stage, NN)   
                                                           9                 (in, IN)   
                                                           11            (halls, NNS)   
                                                           12        (production, NN)   
                                                           13                (of, IN)   
                                                           14           (twelfth, JJ)   
                                                           15             (night, NN)   
                                                           16               (for, IN)   
                                                           17               (the, DT)   
                                                           18          (national, JJ)   
                                                           19           (theatre, NN)   
                                                   1       0                (the, DT)   

                                                                     pos  
review_id                                          sent_id token_id       
Twelfth Night is a New Perspective at the National 0       0          JJ  
                                                           1

In [6]:
LIB['Genre'] = np.where(LIB["Original Work"].isin(['Hamlet','Macbeth', 'Romeo & Juliet', 'Othello', 'Julius Caesar']), 
                        "Tragedy", "Comedy")
LIB['Rating Category'] = np.where(LIB["Overall Rating"].astype(int) > 3, "Positive", "Negative")
LIB = LIB.set_index(["Original Work", "Review Title"])
LIB.head(20)

Review Author  \
Original Work Review Title                                                               
Twelfth Night Twelfth Night is a New Perspective at the National     Christopher Adams   
              Twelfth Night Offers Clarity with Complications     Claudine Nightingale   
              Twelfth Night Goes for Synthesis through Antith...       Denise Battista   
              Shakespeare Set in the Summer of Love: "There's...       Denise Battista   
              Big Things Come in Small Packages                      Carrie Cleaveland   
              The Two Sirs Steal the Show in Twelfth Night        Claudine Nightingale   
              Twelfth Night in Twenties Time Warp                 Claudine Nightingale   
              Always Loving the One You Can't Have                      Roseanne Wells   
              Roses Doth Fall at the Shakespeare Theatre           Georgina Petronella   
              A Twelfth Night Stand You Won't Forget                    Nairi Najarian   
              Malvolio Offers Hope to HSF's Twelfth Night            Cynthia Greenwood   
              Twelfth Night with Bob Dylan at the Helm                 Denise Battista   
              If Music Be the Food of Love, Play On and Eat B...       Melissa Crismon   
              An Enchanting Illyrian Summer in New York                 Roseanne Wells   
              Twelfth Night with Funny Bones But No Guts                Roseanne Wells   
              Decide What You Will in West End’s Twelfth Night    Claudine Nightingale   
              If Music Be the Food of Love, Rock On                     Matthew Barbot   
              Prague Shakespeare Heightens the Antic Side of ...     Cynthia Greenwood   
              A Twelfth Night for Today's Gender Bending Times     Diana Louise Carter   
              Never Bring a Robot to a Fish Fight                      Jennifer Kramer   

                                                                                                            Content  \
Original Work Review Title                                                                                            
Twelfth Night Twelfth Night is a New Perspective at the National  golden brown leaves decorate the edges of the ...   
              Twelfth Night Offers Clarity with Complications      ive been eagerly awaiting the arrival of  dor...   
              Twelfth Night Goes for Synthesis through Antith...  hear ye hear ye!  at stinson beach is now nort...   
              Shakespeare Set in the Summer of Love: "There's...  i was born in the 70s but that doesnt mean i w...   
              Big Things Come in Small Packages                   city lit's production of twelfth night proves ...   
              The Two Sirs Steal the Show in Twelfth Night        everything in twelfth night is about contrast ...   
              Twelfth Night in Twenties Time Warp                 the grounds of london's largest outdoor theatr...   
              Always Loving the One You Can't Have                the hudson river lapped at the dock side amphi...   
              Roses Doth Fall at the Shakespeare Theatre          twelfth night is often considered s last true ...   
              A Twelfth Night Stand You Won't Forget               like a runaway train last metro companys inau...   
              Malvolio Offers Hope to HSF's Twelfth Night         twelfth nights ingenious balance of comedy and...   
              Twelfth Night with Bob Dylan at the Helm            director lesley currier offers a prologue to h...   
              If Music Be the Food of Love, Play On and Eat B...  she removes her black sunglasses and in no rus...   
              An Enchanting Illyrian Summer in New York           every year new yorkers flock to the delacorte ...   
              Twelfth Night with Funny Bones But No Guts          the pearl theatre company is well known for it...   
              Decide What You Will in West End’s Twelfth Night    thi

In [7]:
VOCAB = DOC.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = DOC[['term_str','pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB.head(20)

,n,n_chars,p,i,max_pos
term_str,,,,,
00,1,2,0.000004,18.118926,CD
02138,1,5,0.000004,18.118926,CD
080816,1,6,0.000004,18.118926,CD
081816,1,6,0.000004,18.118926,CD
09,1,2,0.000004,18.118926,CD
1,31,1,0.000109,13.164730,CD
10,4,2,0.000014,16.118926,CD
100,3,3,0.000011,16.533963,CD
1000,1,4,0.000004,18.118926,CD


In [8]:
DOC.to_csv(cwd+"/output/DOC.csv")
LIB.to_csv(cwd+"/output/LIB.csv")
VOCAB.to_csv(cwd+"/output/VOCAB.csv")